In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import os
import time
import json
from jsonmerge import merge

Reading GeoJSON files

In [2]:
with open(r"/home/mohammed/Desktop/Musaddiq/Coursera_Capstone/json/cities.json", 'r', encoding='utf8', errors='ignore') as file:
    cities = json.load(file)
    
with open(r"/home/mohammed/Desktop/Musaddiq/Coursera_Capstone/json/districts.json", 'r', encoding='utf8', errors='ignore') as file:
    districts = json.load(file)

Getting city_id of Al Khobar and Dammam

In [3]:
for city in cities:
    if "Khobar" in city["name_en"]:
        print(str(city["city_id"]) + " " + city["name_en"])
    if "Dammam" in city["name_en"]:
        print(str(city["city_id"]) + " " + city["name_en"])

13 Dammam
31 Al Khobar


Getting distrcits in Al Khobar and Dammam

In [4]:
khobar_districts = []
dammam_districts = []

for district in districts:
    if district["city_id"] == 31:
        khobar_districts.append(district)
        
for district in districts:
    if district["city_id"] == 13:
        dammam_districts.append(district)

Creating a copy in XY Coordinates for GeoJSON

In [5]:
khobar_districts_xy = khobar_districts
dammam_districts_xy = dammam_districts

for district in range(len(khobar_districts_xy)):
    for _ in khobar_districts_xy[district]["boundaries"][0]:
        _.reverse()
        
for district in range(len(dammam_districts_xy)):
    for _ in dammam_districts_xy[district]["boundaries"][0]:
        _.reverse()

In [6]:
khobar = pd.DataFrame(khobar_districts)
dammam = pd.DataFrame(dammam_districts)

khobar_xy = pd.DataFrame(khobar_districts_xy)
dammam_xy = pd.DataFrame(dammam_districts_xy)

khobar_xy

,district_id,city_id,region_id,name_ar,name_en,boundaries
0,10500031001,31,5,حي التحلية,At Tahliyah Dist.,"[[[50.21638279, 26.18209534], [50.21611424, 26..."
1,10500031002,31,5,حي ابن سيناء,Ibn Sina Dist.,"[[[50.21139022, 26.2507773], [50.21288113, 26...."
2,10500031003,31,5,حي الحزام الاخضر,Al Hizam Al Akhdar Dist.,"[[[50.19060022, 26.3089098], [50.19145503, 26...."
3,10500031004,31,5,حي صناعية الثقبة,Sinaiyah Ath Thuqbah Dist.,"[[[50.19749806, 26.24944191], [50.19719675, 26..."
4,10500031005,31,5,حي التعاون,At Taawun Dist.,"[[[50.19458339, 26.23354342], [50.1963495, 26...."
5,10500031006,31,5,حي الراكة الجنوبية,Ar Rakah Al Janubiyah Dist.,"[[[50.20035662, 26.33674744], [50.19986525, 26..."
6,10500031007,31,5,حي الخبر الشمالية,Al Khubar Ash Shamaliyah Dist.,"[[[50.21811139, 26.28045052], [50.21783621, 26..."
7,10500031008,31,5,حي مدينة العمال,Madinat Al Ummal Dist.,"[[[50.20487945, 26.28181475], [50.20483605, 26..."
8,10500031009,31,5,حي العقربية,Al Aqrabiyah Dist.,"[[[50.19759817, 26.28393879], [50.19757992, 26..."
9,10500031010,31,5,حي الخبر الجنوبية,Al Khubar Al Janubiyah Dist.,"[[[50.21598118, 26.26997671], [50.21521935, 26..."


In [85]:
for district in range(len(khobar_districts_xy)):
    feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": khobar_districts_xy[district]["boundaries"]},
            "properties": {
                "district_id": khobar_districts_xy[district]["district_id"],
                "city_id": khobar_districts_xy[district]["city_id"],
                "name_en": khobar_districts_xy[district]["name_en"]}
        }
    
    with open("/home/mohammed/Desktop/Musaddiq/Coursera_Capstone/json/khobar_districts_geojson.json", 'a+') as khobar_districts_geojson:
        string = json.dumps(feature, indent = 2)
        string = string + "," 
        khobar_districts_geojson.write(string)

with open("/home/mohammed/Desktop/Musaddiq/Coursera_Capstone/json/khobar_districts_geojson.json", 'rb+') as khobar_districts_geojson:
        khobar_districts_geojson.seek(-1, os.SEEK_END)
        khobar_districts_geojson.truncate()
    
geojson = {
    "type": "FeatureCollection",
    "features": [json.loads(features)]
}

geojson = json.dumps(geojson)
geojson

JSONDecodeError: Extra data: line 1 column 6352 (char 6351)

In [74]:
count

0

In [75]:
khobar_data = khobar[["name_en", "region_id", "district_id"]]
khobar_data

,name_en,region_id,district_id
0,At Tahliyah Dist.,5,10500031001
1,Ibn Sina Dist.,5,10500031002
2,Al Hizam Al Akhdar Dist.,5,10500031003
3,Sinaiyah Ath Thuqbah Dist.,5,10500031004
4,At Taawun Dist.,5,10500031005
5,Ar Rakah Al Janubiyah Dist.,5,10500031006
6,Al Khubar Ash Shamaliyah Dist.,5,10500031007
7,Madinat Al Ummal Dist.,5,10500031008
8,Al Aqrabiyah Dist.,5,10500031009
9,Al Khubar Al Janubiyah Dist.,5,10500031010


In [76]:
# create a plain map
map = folium.Map(location=[26.2172,50.1971], zoom_start=15)

folium.GeoJson(geojson).add_to(map)

# display map
map